In [ ]:
#| default_exp vacuum

In [ ]:
#| include: false
from fastcore.all import *

In [ ]:
#| export

## State

In [ ]:
#| export
rooms = ["Living Room", "Kitchen", "Office", "Hallway", "Dining Room"]

## Actions

In [ ]:
#| export
action_space = ["L", "R", "U", "D"]
action_spec = """
    1/0/0/0/0 2/8/0/0/0 1/0/0/0/0 2/0/0/8/0
    8/2/0/0/0 0/1/0/0/0 0/1/0/0/0 0/2/0/0/8
    0/0/1/0/0 0/0/2/8/0 0/0/1/0/0 0/0/1/0/0
    0/0/8/2/0 0/0/0/2/8 8/0/0/2/0 0/0/0/1/0
    0/0/0/8/2 0/0/0/0/1 0/8/0/0/2 0/0/0/0/1
    """

## Sensing

In [ ]:
#| export
light_levels = ["dark", "medium", "light"]
sensor_spec = "1/1/8 1/1/8 2/7/1 8/1/1 1/8/1"